<a href="https://colab.research.google.com/github/YoheiFukuhara/nlp100-2020/blob/main/08.%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88/74_%E6%AD%A3%E8%A7%A3%E7%8E%87%E3%81%AE%E8%A8%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

In [1]:
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!python --version
!pip show google tensorflow

Python 3.7.12
Name: google
Version: 2.0.3
Summary: Python bindings to the Google search engine.
Home-page: http://breakingcode.wordpress.com/
Author: Mario Vilas
Author-email: mvilas@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: beautifulsoup4
Required-by: 
---
Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: astunparse, h5py, protobuf, typing-extensions, absl-py, google-pasta, tensorboard, grpcio, keras-preprocessing, termcolor, numpy, clang, flatbuffers, tensorflow-estimator, wheel, gast, keras, six, opt-einsum, wrapt
Required-by: kapre


In [3]:
def _parse_function(example_proto):
    # 特徴の記述
    feature_description = {
        'title': tf.io.FixedLenFeature([], tf.string),
        'category': tf.io.FixedLenFeature([], tf.string)}
  
  # 上記の記述を使って入力の tf.Example を処理
    features = tf.io.parse_single_example(example_proto, feature_description)
    X = tf.io.decode_raw(features['title'], tf.float32)
    y = tf.io.decode_raw(features['category'], tf.int32)
    return X, y

In [4]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks/'

def get_dataset(file_name):
    ds_raw = tf.data.TFRecordDataset(BASE_PATH+file_name+'.tfrecord')
    return ds_raw.map(_parse_function).shuffle(1000).batch(64)

In [5]:
train_ds = get_dataset('train')
test_ds = get_dataset('test')

In [6]:
model = tf.keras.Sequential(
    [tf.keras.layers.Dense(
        4, 
        activation='softmax',
        use_bias=False,
        input_dim=300, 
        kernel_initializer='random_uniform') ])
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 1200      
Total params: 1,200
Trainable params: 1,200
Non-trainable params: 0
_________________________________________________________________


In [7]:
%%time
model.fit(train_ds, epochs=100)

Epoch 1/100
167/167 [==============================] - 2s 8ms/step - loss: 1.3095 - acc: 0.6369
Epoch 2/100
167/167 [==============================] - 1s 8ms/step - loss: 1.1870 - acc: 0.7662
Epoch 3/100
167/167 [==============================] - 1s 8ms/step - loss: 1.1005 - acc: 0.7744
Epoch 4/100
167/167 [==============================] - 1s 8ms/step - loss: 1.0361 - acc: 0.7762
Epoch 5/100
167/167 [==============================] - 1s 8ms/step - loss: 0.9861 - acc: 0.7768
Epoch 6/100
167/167 [==============================] - 1s 8ms/step - loss: 0.9460 - acc: 0.7771
Epoch 7/100
167/167 [==============================] - 1s 8ms/step - loss: 0.9128 - acc: 0.7775
Epoch 8/100
167/167 [==============================] - 1s 8ms/step - loss: 0.8847 - acc: 0.7776
Epoch 9/100
167/167 [==============================] - 1s 8ms/step - loss: 0.8605 - acc: 0.7778
Epoch 10/100
167/167 [==============================] - 1s 8ms/step - loss: 0.8393 - acc: 0.7771
Epoch 11/100
167/167 [=================

In [8]:
model.evaluate(train_ds)

167/167 [==============================] - 2s 8ms/step - loss: 0.4576 - acc: 0.8460


[0.4576301872730255, 0.8460314273834229]

In [9]:
model.evaluate(test_ds)

21/21 [==============================] - 0s 5ms/step - loss: 0.4579 - acc: 0.8488


[0.4579184055328369, 0.848802387714386]